# pre installation

In [ ]:
!pip install qiskit
!pip install qiskit-aer
!pip install qiskit[visualization]
!pip install qiskit-ibm-runtime
!pip install pylatexenc
!pip install qiskit-experiments
!pip install kagglehub[pandas-datasets]

# Import LIB

In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit.circuit.library import ZZFeatureMap
from sklearn.datasets import  load_iris
from qiskit_experiments.library import StateTomography
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from qiskit.quantum_info import Statevector, Operator
from qiskit.circuit.library import QFT,PauliEvolutionGate,QFT
from qiskit.visualization import plot_histogram
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import PhaseEstimation
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.datasets import  load_iris
from sklearn.decomposition import PCA
import kagglehub
from kagglehub import KaggleDatasetAdapter
import pandas as pd
from sklearn.datasets import fetch_california_housing, load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


# DATA

In [ ]:
data = pd.read_csv('https://datacatalogfiles.worldbank.org/ddh-published/0066034/10/DR0094183/global_biod_species_occ_endemism_and_small_range.csv')

In [ ]:
data.columns

In [ ]:
data.shape

In [ ]:
data.dropna(inplace=True)

In [ ]:
y = data['type'].values
data = data.drop('type', axis=1)

In [ ]:
y.reshape(-1,1)

In [ ]:
y = pd.factorize(y)[0]

In [ ]:
y_cls = y

In [ ]:
col = data.columns
lam = LabelEncoder()
for i in col:
    data[i] = lam.fit_transform(data[i])

In [ ]:
X_cls = data

In [ ]:
X_cls.shape

In [ ]:
pca = PCA(n_components=4)
X_cls = pca.fit_transform(X_cls)

In [ ]:
X_cls.shape

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X_cls, y_cls, test_size=0.2, random_state=42)

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
X_normalized = scaler.fit_transform(x_train)
X_test_normalized = scaler.transform(x_test)
Y_normalized = scaler.fit_transform(y_train.reshape(-1, 1))
Y_test_normalized = scaler.transform(y_test.reshape(-1, 1))
num_dem = X_normalized.shape[1]

In [ ]:
num_target_dim=2

In [ ]:
num_samples, num_features = X_normalized.shape
next_power_of_two = 2 ** int(np.ceil(np.log2(num_features)))
num_padding = next_power_of_two - num_features
global_mean = np.mean(X_normalized)
X_normalized = np.pad(X_normalized, ((0,0), (0, num_padding)), mode='constant', constant_values=global_mean)
num_dem = X_normalized.shape[1]

In [ ]:
X_test_normalized = np.pad(X_test_normalized, ((0,0), (0, num_padding)),
                    mode='constant',
                    constant_values=global_mean)

In [ ]:
print(num_dem)

# Circuit

In [ ]:
# Initial settings
num_samples = X_normalized.shape[0]
num_features = X_normalized.shape[1]
num_qubits = int(np.log2(num_features))
num_eval_qubits =num_target_dim  # Estimated number of qubits
feature_map = ZZFeatureMap(feature_dimension=num_qubits, reps=1, entanglement='linear')
qpe = QuantumCircuit(num_qubits + num_eval_qubits, num_eval_qubits)

In [ ]:
qpe.barrier()
qpe.draw('mpl')

In [ ]:
# Adding Feature Map and Estimation Qubits
qpe.append(feature_map, range(num_qubits))
for qubit in range(num_eval_qubits):
    qpe.h(num_qubits + qubit)

In [ ]:
qpe.barrier()
qpe.draw('mpl')

In [ ]:
# Applying the time evolution operator
hamiltonian = SparsePauliOp.from_operator(np.cov(X_normalized.T) / np.trace(np.cov(X_normalized.T)))  # Density matrix
t = 2 * np.pi
for qubit in range(num_eval_qubits):
    exponent = 2 ** (num_eval_qubits - qubit - 1)
    evolution_gate = PauliEvolutionGate(hamiltonian, time=-t * exponent)
    ctrl_evolution = evolution_gate.control(1)
    qpe.append(ctrl_evolution, [num_qubits +qubit ] +list(range(num_qubits)))

In [ ]:
qpe.barrier()
qpe.draw('mpl')

In [ ]:
# Inverse QFT and measurement
qpe.append(QFT(num_eval_qubits, inverse=True), range(num_qubits, num_qubits + num_eval_qubits))
qpe.measure(range(num_qubits, num_qubits + num_eval_qubits), range(num_eval_qubits))

In [ ]:
qpe.barrier()
qpe.draw('mpl')

# Assign Parameters

In [ ]:
all_eigvecs = []
i =0
for sample_idx in range(num_samples//8):
    if i % 100 == 0 :
      print(i)
    sample = X_normalized[sample_idx]

    # Assigning parameters to the sample
    parameter_binds = {param: sample[i % num_qubits] for i, param in enumerate(feature_map.parameters)}
    bound_qpe = qpe.assign_parameters(parameter_binds)

    # Separating data qubits
    data_circuit = QuantumCircuit(num_qubits )
    for instruction in bound_qpe.data:
        if all(qubit in bound_qpe.qubits[:num_qubits] for qubit in instruction.qubits):
            qubit_indices = [bound_qpe.qubits.index(q) for q in instruction.qubits]
            data_circuit.append(instruction.operation, qubit_indices)

    # Transpilation of the circuit for compatibility with the state vector simulator
    simulator = Aer.get_backend('statevector_simulator')
    transpiled_circuit = transpile(data_circuit, simulator)

    # Implementation of the piled transformer circuit
    result = simulator.run(transpiled_circuit).result()
    statevector = result.get_statevector()

    # Constructing a density matrix from a quantum state
    rho_estimated = np.outer(statevector.data, np.conj(statevector.data))
    rho_estimated = (rho_estimated + rho_estimated.T.conj()) / 2

    # Extracting eigenvectors
    eigvals, eigvecs = np.linalg.eigh(rho_estimated)
    all_eigvecs.append(eigvecs)
    i += 1

In [ ]:
qpe.barrier()
qpe.draw('mpl')

# Vectors

In [ ]:
# Averaging of eigenvectors
avg_eigvecs = np.mean(all_eigvecs, axis=0)
top_eigvecs = avg_eigvecs.real[:, :num_target_dim]

# Decompose

In [ ]:
# Data dimensionality reduction with eigenvector averaging
X_qpca = X_normalized[:55700] @ top_eigvecs

In [ ]:
print("Reduced data with QPCA (Averaged):\n", X_qpca)

# **compair with PCA**

In [ ]:
pca = PCA(n_components=num_target_dim)
X_reduced = pca.fit_transform(X_normalized[:55700])

# SVC MODEL(PCA,QPCA)

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report
from sklearn.preprocessing import LabelBinarizer

In [ ]:
X_test_pca = pca.transform(X_test_normalized)

In [ ]:
X_test_qpca = X_test_normalized @ top_eigvecs

In [ ]:
svm_pca = SVC(probability=True).fit(X_reduced, y_train.ravel()[:55700])

In [ ]:
svm_qpca = SVC(probability=True).fit(X_qpca, y_train.ravel()[:55700])

In [ ]:
y_pred_pca = svm_pca.predict(X_test_pca)

In [ ]:
y_pred_qpca = svm_qpca.predict(X_test_qpca)

In [ ]:
# 4. Calculating evaluation criteria
# --------------------------------------------------
def calculate_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='weighted')

    # Calculating AUC for multiple classes
    lb = LabelBinarizer()
    y_true_bin = lb.fit_transform(y_true)
    y_pred_bin = lb.transform(y_pred)

    auc = roc_auc_score(y_true_bin, y_pred_bin, multi_class='ovo')
    return accuracy, f1, auc

In [ ]:
# Calculation for PCA
acc_pca, f1_pca, auc_pca = calculate_metrics(y_test.ravel(), y_pred_pca)

In [ ]:
# Calculation for QPCA
acc_qpca, f1_qpca, auc_qpca = calculate_metrics(y_test.ravel(), y_pred_qpca)

In [ ]:
# 5. Show results
# --------------------------------------------------
print("\n" + "="*40)
print("Comparing the performance of PCA and QPCA in classification")
print("="*40)
print(f"{'Metric':<15} | {'PCA':<10} | {'QPCA':<10}")
print("-"*40)
print(f"{'Accuracy':<15} | {acc_pca:.4f}    | {acc_qpca:.4f}")
print(f"{'F1-Score':<15} | {f1_pca:.4f}    | {f1_qpca:.4f}")
print(f"{'AUC-OVO':<15} | {auc_pca:.4f}    | {auc_qpca:.4f}")
print("="*40)

# Full classification report
print("\nClassification report for PCA:")
print(classification_report(y_test, y_pred_pca))

print("\nClassification report for QPCA:")
print(classification_report(y_test, y_pred_qpca))

metrics = ['Accuracy', 'F1-Score', 'AUC-OVO']
pca_scores = [acc_pca, f1_pca, auc_pca]
qpca_scores = [acc_qpca, f1_qpca, auc_qpca]

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(metrics, pca_scores, 'o-', label='PCA', color='blue', markersize=8)
plt.plot(metrics, qpca_scores, 's--', label='QPCA', color='red', alpha=0.7)
plt.xlabel('Evaluation criteria', fontsize=20)
plt.ylabel('Amount', fontsize=20)
plt.title('Comparing the performance of PCA and QPCA in classification', fontsize=20)
plt.ylim(0, 1.5)
plt.grid(True, linestyle='--', alpha=0.5)
plt.legend()
plt.show()

# PLOT

In [ ]:
plt.figure(figsize=(18, 6))

plt.scatter(X_qpca[:, 0], X_qpca[:, 1],c=Y_normalized[:55700],cmap='viridis', edgecolor='k', s=50)
plt.title(f'Qpca Visualization')
plt.xlabel('Component 1')
plt.ylabel('Component 2')

plt.tight_layout()
plt.show()

In [ ]:
pca = PCA(n_components=4)

plt.figure(figsize=(18, 6))

X_reduced = pca.fit_transform(X_normalized)

plt.scatter(X_reduced[:55700, 0], X_reduced[:55700, 1], c=Y_normalized[:55700],
            cmap='viridis', edgecolor='k', s=50)
plt.title(f'pca Visualization')
plt.xlabel('Component 1')
plt.ylabel('Component 2')

plt.tight_layout()
plt.show()

In [ ]:

from sklearn.metrics import (mean_absolute_error, mean_squared_error, r2_score,accuracy_score, precision_score, recall_score, f1_score,confusion_matrix)
import seaborn as sns
# Classification evaluation
print("\nSVC-PCA:")
print(f"Accuracy: {acc_pca:.2f}")
print(f"Precision: {auc_pca:.2f}")
# print(f"Recall: {recall_score(y_cls_test, y_cls_pred_logreg, average='macro'):.2f}")
print(f"F1-Score: {f1_pca:.2f}")

print("\nSVC-QPCA:")
print(f"Accuracy: {acc_qpca:.2f}")
print(f"Precision: {auc_qpca:.2f}")
# print(f"Recall: {recall_score(y_cls_test, y_cls_pred_svc, average='macro'):.2f}")
print(f"F1-Score: {f1_qpca:.2f}")

# Confusion matrices
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
sns.heatmap(confusion_matrix(y_test.ravel(), y_pred_pca),
            annot=True, fmt='d', cmap='Blues')
plt.title('SVC-PCA')
plt.xlabel('Predicted')
plt.ylabel('Actual')

plt.subplot(1, 2, 2)
sns.heatmap(confusion_matrix(y_test.ravel(),y_pred_qpca),
            annot=True, fmt='d', cmap='Blues')
plt.title('SVC-QPCA')
plt.xlabel('Predicted')
plt.ylabel('Actual')

plt.tight_layout()
plt.show()

In [ ]:
import time
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA
from qiskit.quantum_info import state_fidelity

# 1. RMSE between original and reconstructed data
def calculate_rmse(original, reconstructed):
    return np.sqrt(mean_squared_error(original, reconstructed))

# 2. EVR (Explained Variance Ratio)
def explained_variance_ratio(X_original, X_reduced, components):
    X_reconstructed = X_reduced @ components.T
    total_variance = np.var(X_original, axis=0).sum()
    explained_variance = np.var(X_reconstructed, axis=0).sum()
    return explained_variance / total_variance

# 3. CR (Compression Ratio)
def compression_ratio(original_dim, compressed_dim):
    return original_dim / compressed_dim

# 4. execution time
def measure_execution_time(func):
    start_time = time.time()
    result = func()
    end_time = time.time()
    return result, end_time - start_time

# 5. Quantum Fidelity
def quantum_fidelity(state1, state2):
    return state_fidelity(state1, state2)

# 6. Classification Accuracy (Calculated in the previous section)

In [ ]:
# 1. RMSE for PCA and QPCA
X_reconstructed_pca = X_reduced @ pca.components_
X_reconstructed_qpca = X_qpca @ top_eigvecs.T

rmse_pca = calculate_rmse(X_normalized[:55700], X_reconstructed_pca[:55700])
rmse_qpca = calculate_rmse(X_normalized[:55700], X_reconstructed_qpca[:55700])

# 2. EVR for PCA and QPCA
evr_pca = explained_variance_ratio(X_normalized, X_reduced, pca.components_)
evr_qpca = explained_variance_ratio(X_normalized, X_qpca, top_eigvecs)

# 3. CR
cr_pca = compression_ratio(X_normalized.shape[1], X_reduced.shape[1])
cr_qpca = compression_ratio(X_normalized.shape[1], X_qpca.shape[1])

# 4. execution time  (Example: Conversion time  PCA)
_, exec_time_pca = measure_execution_time(lambda: pca.transform(X_test_normalized))
_, exec_time_qpca = measure_execution_time(lambda: X_test_normalized @ top_eigvecs)

# 5. Quantum Fidelity (Between two quantum states)
# Each sample is assumed to have a quantum state.
# For example, the first example of PCA and QPCA
statevector_simulator = Aer.get_backend('statevector_simulator')

#  PCA mode (Simulation as a quantum state)
qc_pca = QuantumCircuit(num_qubits)
for i in range(num_qubits):
    qc_pca.ry(X_normalized[0, i], i)
state_pca = Statevector(qc_pca).evolve(transpile(qc_pca, simulator))

#  QPCA mode
qc_qpca = QuantumCircuit(num_qubits)
for i in range(num_qubits):
    qc_qpca.ry(X_qpca[0, i % num_target_dim], i)
state_qpca = Statevector(qc_qpca).evolve(transpile(qc_qpca, simulator))

fid = quantum_fidelity(state_pca, state_qpca)

In [ ]:
# Show results
print("\n" + "=" * 40)
print("Evaluation Metrics for PCA and QPCA")
print("=" * 40)
print(f"{'Metric':<20} | {'PCA':<10} | {'QPCA':<10}")
print("-" * 40)
print(f"{'RMSE':<20} | {rmse_pca:.4f}   | {rmse_qpca:.4f}")
print(f"{'EVR':<20} | {evr_pca:.4f}   | {evr_qpca:.4f}")
print(f"{'CR':<20} | {cr_pca:.4f}   | {cr_qpca:.4f}")
print(f"{'Execution Time (s)':<20} | {exec_time_pca:.4f}   | {exec_time_qpca:.4f}")
print(f"{'Quantum Fidelity':<20} | {'-':<10} | {fid:.4f}")
print("=" * 40)

In [ ]:
# Calculating temporal RMSE for PCA and QPCA
rmse_temporal_pca = []
rmse_temporal_qpca = []

for i in range(len(X_normalized[:55700])):
    # PCA and QPCA data reconstruction
    X_reconstructed_pca = X_reduced[i] @ pca.components_
    X_reconstructed_qpca = X_qpca[i] @ top_eigvecs.T

    # Calculate RMSE for each sample
    rmse_pca = np.sqrt(mean_squared_error(X_normalized[i], X_reconstructed_pca))
    rmse_qpca = np.sqrt(mean_squared_error(X_normalized[i], X_reconstructed_qpca))

    rmse_temporal_pca.append(rmse_pca)
    rmse_temporal_qpca.append(rmse_qpca)

# Plotting the RMSE over time graph
plt.figure(figsize=(12, 6))
plt.plot(rmse_temporal_pca, label='PCA', color='blue', alpha=0.7)
plt.plot(rmse_temporal_qpca, label='QPCA', color='red', alpha=0.7)
plt.title('Temporal RMSE')
plt.xlabel('Time Step')
plt.ylabel('RMSE')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Cumulative EVR for PCA
explained_variance_pca = pca.explained_variance_ratio_
cumulative_evr_pca = np.cumsum(explained_variance_pca)

# Cumulative EVR for QPCA (Assumption: top_eigvecs has eigenvectors)
# Calculating EVR for QPCA
explained_variance_qpca = np.var(X_qpca, axis=0) / np.var(X_normalized[:55700], axis=0).sum()
cumulative_evr_qpca = np.cumsum(explained_variance_qpca)

# Plotting the cumulative EVR chart
plt.figure(figsize=(10, 6))
plt.plot(cumulative_evr_pca, 'o-', label='PCA', color='blue')
plt.plot(range(num_target_dim), cumulative_evr_qpca, 's--', label='QPCA', color='red')
plt.title('Cumulative Explained Variance Ratio')
plt.xlabel('Components')
plt.ylabel('Cumulative EVR')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Comparison of PCA and QPCA latent space
plt.figure(figsize=(12, 6))

# PCA
plt.subplot(1, 2, 1)
plt.scatter(X_reduced[:55700, 0], X_reduced[:55700, 1], c=y_train[:55700], cmap='viridis', edgecolor='k', s=50)
plt.title('PCA Latent Space')
plt.xlabel('Component 1')
plt.ylabel('Component 2')

# QPCA
plt.subplot(1, 2, 2)
plt.scatter(X_qpca[:55700, 0], X_qpca[:55700, 1], c=y_train[:55700], cmap='viridis', edgecolor='k', s=50)
plt.title('QPCA Latent Space')
plt.xlabel('Component 1')
plt.ylabel('Component 2')

plt.tight_layout()
plt.show()

In [ ]:
# Assumption: computational_cost_pca و computational_cost_qpca Calculated from execution time
# Calculating CR
cr_pca = compression_ratio(X_normalized.shape[1], X_reduced.shape[1])
cr_qpca = compression_ratio(X_normalized.shape[1], X_qpca.shape[1])

# Plotting the CR vs Computational Cost graph
plt.figure(figsize=(8, 5))
plt.scatter([cr_pca], [exec_time_pca], color='blue', label='PCA', s=100)
plt.scatter([cr_qpca], [exec_time_qpca], color='red', label='QPCA', s=100)
plt.xlabel("Compression Ratio (CR)")
plt.ylabel("Computational Cost (Execution Time)")
plt.title("CR vs Computational Cost")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Runtime comparison
plt.figure(figsize=(8, 5))
plt.bar(['PCA', 'QPCA'], [exec_time_pca, exec_time_qpca], color=['blue', 'red'])
plt.ylabel('Execution Time (s)')
plt.title('Execution Time Comparison')
plt.grid(True)
plt.show()

In [ ]:
# Plotting CR vs RMSE
plt.figure(figsize=(8, 5))
plt.scatter([cr_pca], [rmse_pca], color='blue', label='PCA', s=100)
plt.scatter([cr_qpca], [rmse_qpca], color='red', label='QPCA', s=100)
plt.xlabel("Compression Ratio (CR)")
plt.ylabel("RMSE")
plt.title("CR vs RMSE")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Calculate Quantum Fidelity for all samples
fidelities = []
for i in range(min(10, len(X_normalized[:55700]))):  # Only the first 10 samples for speed
    # PCA mode
    qc_pca = QuantumCircuit(num_qubits)
    for j in range(num_qubits):
        qc_pca.ry(X_normalized[i, j], j)
    state_pca = Statevector(qc_pca).evolve(transpile(qc_pca, simulator))

    # QPCA mode
    qc_qpca = QuantumCircuit(num_qubits)
    for j in range(num_qubits):
        qc_qpca.ry(X_qpca[i, j % num_target_dim], j)
    state_qpca = Statevector(qc_qpca).evolve(transpile(qc_qpca, simulator))

    # Fidelity calculation
    fid = quantum_fidelity(state_pca, state_qpca)
    fidelities.append(fid)

# Quantum Fidelity Drawing
plt.figure(figsize=(10, 6))
plt.plot(fidelities, 'o-', color='green')
plt.title('Quantum Fidelity Over Samples')
plt.xlabel('Sample Index')
plt.ylabel('Fidelity')
plt.grid(True)
plt.show()

In [ ]:
# Comparing classification accuracy
plt.figure(figsize=(8, 5))
plt.bar(['PCA', 'QPCA'], [acc_pca, acc_qpca], color=['blue', 'red'])
plt.ylabel('Accuracy')
plt.title('Classification Accuracy Comparison')
plt.grid(True)
plt.show()

In [ ]:
list1 = []
list1.extend(x for x in range(10))

In [ ]:
list1